k-최근접 이웃 알고리즘: 테스트 데이터랑 가장 가까운 데이터 n개를 바탕으로 어디에 속하는지 알아냄 (from ch.1)<p>
가장 가까운 샘플 몇 개를 구해서 이를 확률로 삼을 수 있음

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from scipy.special import expit
from scipy.special import softmax

데이터 준비

In [ ]:
fish = pd.read_csv('https://bit.ly/fish_csv_data')
fish.head()   #head(): 처음 5개 행 출력
fish_input = fish[['Weight','Length','Diagonal','Height','Width']].to_numpy()
fish_target = fish['Species'].to_numpy()
train_input, test_input, train_target, test_target = train_test_split(fish_input, fish_target, random_state=42)

ss = StandardScaler()
ss.fit(train_input)
train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)

In [ ]:
print(pd.unique(fish['Species']))             #unique(): 열에서 고유한 값 출력

타깃 데이터에 2개 이상의 클래스가 포함된 문제: **다중 분류**

In [ ]:
kn = KNeighborsClassifier(n_neighbors=3)
kn.fit(train_scaled, train_target)
#print(kn.score(train_scaled, train_target))
#print(kn.score(test_scaled, test_target))
print(kn.classes_)
print(kn.predict(test_scaled[:5]))
proba = kn.predict_proba(test_scaled[:5]) #클래스별 확률값 반환
print(np.round(proba,decimals=4))

['Bream' 'Parkki' 'Perch' 'Pike' 'Roach' 'Smelt' 'Whitefish']
['Perch' 'Smelt' 'Pike' 'Perch' 'Perch']
[[0.     0.     1.     0.     0.     0.     0.    ]
 [0.     0.     0.     0.     0.     1.     0.    ]
 [0.     0.     0.     1.     0.     0.     0.    ]
 [0.     0.     0.6667 0.     0.3333 0.     0.    ]
 [0.     0.     0.6667 0.     0.3333 0.     0.    ]]


문제점: 3개의 최근접 이웃만을 사용하므로 변수가 적음<p><p>
로지스틱 회귀: 선형 회귀와 동일하게 선형 방정식을 학습하되, 0~1 사이의 값을 리턴<p>
$z$ = $a_1$*(Data1) + $a_2$*(Data2) + ... + $a_n$ <p>
시그모이드 함수<p>
$\emptyset$ = $\dfrac{1}{1+e^{-z}}$<p>


In [ ]:
#시그모이드 함수 그래프
z = np.arange(-5,5,0.1)
phi = 1 / (1+np.exp(-z))
plt.plot(z,phi)
plt.show()

불리언 인덱싱: True, False 값을 전달하여 행 선택

In [ ]:
#불리언 인덱싱
arr = np.array([1,2,3,4])
print(arr[[True,False,False,True]])

[1 4]


이진 분류: ∅ = ?<p>음성 클래스 < 0.5 < 양성 클래스<p>

In [ ]:
bream_smelt_indexes = (train_target == 'Bream') | (train_target == 'Smelt') #Bream or Smelt 행만 골라내기 [T,T,F,F,...]
train_bream_smelt = train_scaled[bream_smelt_indexes]
target_bream_smelt = train_target[bream_smelt_indexes]

lr = LogisticRegression()
lr.fit(train_bream_smelt,target_bream_smelt)
decisions = lr.decision_function(train_bream_smelt[:5]) #로지스틱 회귀에 따른 z값

print(lr.predict(train_bream_smelt[:5]))
print(lr.classes_ ,end = " 음성, 양성\n")
print(lr.predict_proba(train_bream_smelt[:5]))
print(lr.coef_, lr.intercept_)
print(decisions)
print(expit(decisions)) #양성 확률값 출력

['Bream' 'Smelt' 'Bream' 'Bream' 'Bream']
['Bream' 'Smelt']음성, 양성
[[0.99759855 0.00240145]
 [0.02735183 0.97264817]
 [0.99486072 0.00513928]
 [0.98584202 0.01415798]
 [0.99767269 0.00232731]]
[[-0.4037798  -0.57620209 -0.66280298 -1.01290277 -0.73168947]] [-2.16155132]
[-6.02927744  3.57123907 -5.26568906 -4.24321775 -6.0607117 ]
[0.00240145 0.97264817 0.00513928 0.01415798 0.00232731]


로지스틱 회귀에서의 다중 분류:<p>
[C] 계수의 제곱 규제, C가 작을수록 규제가 커짐, 기본값 1<p>
[max_iter] 반복 횟수 설정, 기본값 100<p>
<p>

In [ ]:
lr = LogisticRegression(C = 20, max_iter = 1000)
lr.fit(train_scaled, train_target)
proba = lr.predict_proba(test_scaled[:5])

print(lr.score(train_scaled, train_target))
print(lr.score(test_scaled, test_target))
print(lr.predict(test_scaled[:5]))
print(lr.classes_)
print(np.round(proba,decimals = 3))
print(lr.coef_.shape, lr.intercept_.shape)

0.9327731092436975
0.925
['Perch' 'Smelt' 'Pike' 'Roach' 'Perch']
['Bream' 'Parkki' 'Perch' 'Pike' 'Roach' 'Smelt' 'Whitefish']
[[0.    0.014 0.841 0.    0.136 0.007 0.003]
 [0.    0.003 0.044 0.    0.007 0.946 0.   ]
 [0.    0.    0.034 0.935 0.015 0.016 0.   ]
 [0.011 0.034 0.306 0.007 0.567 0.    0.076]
 [0.    0.    0.904 0.002 0.089 0.002 0.001]]
(7, 5) (7,)


클래스마다 z값을 하나씩 계산 -> 소프트맥스 함수 사용 <p>
$e_{sum} = e^{z1} + e^{z2} + ... + e^{z7}$ <p>
$s1 = \dfrac{e^{z1}}{e_{sum}}, s2 = \dfrac{e^{z2}}{e_{sum}}, ...$<p>
axis: 소프트맥스를 계산할 축 지정

In [ ]:
decision = lr.decision_function(test_scaled[:5])
proba = softmax(decision, axis=1)
print(np.round(decision, decimals=2))
print(np.round(proba,decimals=3))

[[ -6.5    1.03   5.16  -2.73   3.34   0.33  -0.63]
 [-10.86   1.93   4.77  -2.4    2.98   7.84  -4.26]
 [ -4.34  -6.23   3.17   6.49   2.36   2.42  -3.87]
 [ -0.68   0.45   2.65  -1.19   3.26  -5.75   1.26]
 [ -6.4   -1.99   5.82  -0.11   3.5   -0.11  -0.71]]
[[0.    0.014 0.841 0.    0.136 0.007 0.003]
 [0.    0.003 0.044 0.    0.007 0.946 0.   ]
 [0.    0.    0.034 0.935 0.015 0.016 0.   ]
 [0.011 0.034 0.306 0.007 0.567 0.    0.076]
 [0.    0.    0.904 0.002 0.089 0.002 0.001]]
